Para esta atividade iremos usar um conjunto de dados sobre preços de imóveis. O conjunto original pode ser encontrado no [kaggle](https://www.kaggle.com/datasets/gunhee/koreahousedata). Os dados que usaremos foram alterados para facilitar as análises.

# Regressão Linear Simples

## Atividade 1: Importe os dados e verifique se há dados faltantes.

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

## Atividade 2: Selecione apenas as variáveis `preco` e `m2`, aplique o logaritmo natural na variável `m2` e verifique a correlação entre as variáveis.

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

## Atividade 3: Crie um modelo de regressão linear simples e encontre os coeficientes do modelo.

In [ ]:
# seu código

In [ ]:
# seu código

## Atividade 4: Verifique os pressupostos da regressão linear.

In [ ]:
# seu código

Linearidade

In [ ]:
# seu código

Homocedasticidade

Vamos checar a homocedasticidade com um teste de hipótese: Goldfeld Quandt. Este teste pode ser realizado importando-se a função `statsmodels.stats.api.het_goldfeldquandt()`.

Neste teste, a hipótese nula é de que há homocedasticidade.

In [ ]:
# seu código

Normalidade dos Resíduos

Vamos checar a normalidadde dos resíduos usando o teste de hipóteses de Shapiro-Wilk. Neste caso, a hipótese nula é que a amostra segue distribuição normal. Podemos realizar este teste importando a função `scipy.stats.shapiro()`.

In [ ]:
# seu código

Autocorrelação dos Resíduos

Para checar se há autocorrelação entre os resíduos vamos usar o teste de hipótese de Durbin Watson. Ele pode ser importado do `statsmodels` com a sintaxe `statsmodels.stats.stattolls.durbin_watons()`.

In [ ]:
# seu código

Para decidirmor se há ou não autocorrelação dos resíduos, devemos observar o valor do teste. Valores entre 1.5 e 2.5 indicam que não há autocorrelação, valores fora dessa faixa indicam que há autocorrelação.

Multicolinearidade



In [ ]:
# seu código

# Regressão Linear Múltipla

## Atividade 5: Crie um modelo de regressão linear múltipla.

Para facilitar o trabalho, vamos deixar um pré processamento pronto.

In [ ]:
# importando os dados
df = pd.read_excel('apartamentos.xlsx')
df.head()

,preco,ano_construcao,ano_Venda,m2,andar,tipo_Aquecimento,tempo_ate_onibus,tempo_ate_metro,qtde_elevadores,qtde_escritorios_publicos_perto,qtde_hospitais_perto,qtde_lojas_departamento_perto,qtde_shoppings_perto,qtde_estacionamentos_perto,qtde_escolas_fundamental1_perto,qtde_escolas_fundamental2_perto,qtde_escolas_medio_perto,qtde_universidades_perto
0,849552,2006,2007,75.623042,3,individual,5min~10min,10min~15min,0,2,1,1,1,0,3,2,2,2
1,307962,1985,2007,54.534061,8,individual,0~5min,5min~10min,2,5,1,2,1,1,2,1,1,0
2,292032,1985,2007,54.534061,6,individual,0~5min,5min~10min,2,5,1,2,1,1,2,1,1,0
3,2283180,2006,2007,191.008568,8,individual,0~5min,0-5min,11,1,1,0,1,0,2,2,1,2
4,1327428,1993,2007,163.602183,3,individual,0~5min,15min~20min,20,6,2,0,1,0,4,3,5,5


In [ ]:
# Vamos condensar algumas variáveis em uma única
df['qtde_servicos'] = df['qtde_hospitais_perto'] + \
                      df['qtde_lojas_departamento_perto'] + \
                      df['qtde_escritorios_publicos_perto'] + \
                      df['qtde_escolas_fundamental1_perto'] + \
                      df['qtde_escolas_fundamental2_perto'] + \
                      df['qtde_escolas_medio_perto'] + \
                      df['qtde_universidades_perto']

# excluindo as variáveis originais do DataFrame
df.drop(['qtde_hospitais_perto','qtde_lojas_departamento_perto',
         'qtde_escritorios_publicos_perto','qtde_escolas_fundamental1_perto',
         'qtde_escolas_fundamental2_perto','qtde_escolas_medio_perto',
         'qtde_universidades_perto'],
        axis=1, inplace=True)

In [ ]:
# criando nova variável
df['qtde_conveniencia'] = df['qtde_shoppings_perto'] + df['qtde_estacionamentos_perto']

# excluindo variáveis do DataFrame
df.drop(['qtde_shoppings_perto','qtde_estacionamentos_perto'],
        axis=1, inplace=True)

df.head()

,preco,ano_construcao,ano_Venda,m2,andar,tipo_Aquecimento,tempo_ate_onibus,tempo_ate_metro,qtde_elevadores,qtde_servicos,qtde_conveniencia
0,849552,2006,2007,75.623042,3,individual,5min~10min,10min~15min,0,13,1
1,307962,1985,2007,54.534061,8,individual,0~5min,5min~10min,2,12,2
2,292032,1985,2007,54.534061,6,individual,0~5min,5min~10min,2,12,2
3,2283180,2006,2007,191.008568,8,individual,0~5min,0-5min,11,9,1
4,1327428,1993,2007,163.602183,3,individual,0~5min,15min~20min,20,25,1


In [ ]:
# aplicando One Hot Encoding à variável tipo_Aquecimento
df_aux = pd.get_dummies(df['tipo_Aquecimento'], prefix='tipo_Aquecimento')
df = pd.concat([df, df_aux], axis=1)
df.drop(['tipo_Aquecimento', 'tipo_Aquecimento_central'], axis=1, inplace=True)

In [ ]:
# aplicando One Hot Encoding à variável tempo_ate_onibus
df_aux = pd.get_dummies(df['tempo_ate_onibus'], prefix='tempo_ate_onibus')
df = pd.concat([df, df_aux], axis=1)
df.drop(['tempo_ate_onibus', 'tempo_ate_onibus_10min~15min'], axis=1, inplace=True)

In [ ]:
# aplicando One Hot Encoding à variável tempo_ate_metro
df_aux = pd.get_dummies(df['tempo_ate_metro'], prefix='tempo_ate_metro')
df = pd.concat([df, df_aux], axis=1)
df.drop(['tempo_ate_metro', 'tempo_ate_metro_no_bus_stop_nearby'], axis=1, inplace=True)
df.head()

,preco,ano_construcao,ano_Venda,m2,andar,qtde_elevadores,qtde_servicos,qtde_conveniencia,tipo_Aquecimento_individual,tempo_ate_onibus_0~5min,tempo_ate_onibus_5min~10min,tempo_ate_metro_0-5min,tempo_ate_metro_10min~15min,tempo_ate_metro_15min~20min,tempo_ate_metro_5min~10min
0,849552,2006,2007,75.623042,3,0,13,1,1,0,1,0,1,0,0
1,307962,1985,2007,54.534061,8,2,12,2,1,1,0,0,0,0,1
2,292032,1985,2007,54.534061,6,2,12,2,1,1,0,0,0,0,1
3,2283180,2006,2007,191.008568,8,11,9,1,1,1,0,1,0,0,0
4,1327428,1993,2007,163.602183,3,20,25,1,1,1,0,0,0,1,0


In [ ]:
# separando os dados em X e y
X = df.drop('preco', axis=1)
y = df['preco']

In [ ]:
# importando StandardScaler
from sklearn.preprocessing import StandardScaler

# instanciando a função
scaler = StandardScaler()

# criando uma cópia dos valores
X_scaled = X.copy()

# aplicando a padronização
X_scaled[X_scaled.columns] = scaler.fit_transform(X_scaled)

X_scaled.head()

,ano_construcao,ano_Venda,m2,andar,qtde_elevadores,qtde_servicos,qtde_conveniencia,tipo_Aquecimento_individual,tempo_ate_onibus_0~5min,tempo_ate_onibus_5min~10min,tempo_ate_metro_0-5min,tempo_ate_metro_10min~15min,tempo_ate_metro_15min~20min,tempo_ate_metro_5min~10min
0,0.344181,-1.959067,-0.370182,-1.195839,-1.427953,-0.723875,-0.772684,0.231641,-1.806284,1.854545,-0.938566,2.511759,-0.439310,-0.488514
1,-2.039194,-1.959067,-0.963752,-0.533420,-1.171726,-0.896304,0.524518,0.231641,0.553623,-0.539216,-0.938566,-0.398127,-0.439310,2.047024
2,-2.039194,-1.959067,-0.963752,-0.798388,-1.171726,-0.896304,0.524518,0.231641,0.553623,-0.539216,-0.938566,-0.398127,-0.439310,2.047024
3,0.344181,-1.959067,2.877458,-0.533420,-0.018703,-1.413592,-0.772684,0.231641,0.553623,-0.539216,1.065455,-0.398127,-0.439310,-0.488514
4,-1.131242,-1.959067,2.106078,-1.195839,1.134320,1.345276,-0.772684,0.231641,0.553623,-0.539216,-0.938566,-0.398127,2.276298,-0.488514


In [ ]:
# seu código

In [ ]:
# seu código

## Atividade 6: Faça a previsão do preço de um apartamento com as seguinte características:
ano_construcao = 2000

ano_Venda = 2003 

m2 = 83

andar = 2

qtde_elevadores = 1

qtde_servicos = 4

qtde_conveniencia = 2

tipo_Aquecimento_individual = 1

tempo_ate_onibus_0~5min = 0

tempo_ate_onibus_5min~10min = 1

tempo_ate_metro_0-5min = 1

tempo_ate_metro_10min~15min = 0

tempo_ate_metro_15min~20min = 0

tempo_ate_metro_5min~10min = 0

In [ ]:
# seu código

## Atividade 7: Verifique os pressupostos do modelo.

In [ ]:
# seu código

Homocedasticidade dos Resíduos

In [ ]:
# seu código

Normalidade dos Reíduos

In [ ]:
# seu código

Autocorrelação dos Resíduos

In [ ]:
# seu código

Multicolinearidade

In [ ]:
# seu código

## Atividade 7: Avalie e compare ambos os modelos e decida qual é melhor.

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código

In [ ]:
# seu código